In [1]:
import pandas as pd 
import tensorflow as tf 
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import matplotlib.pyplot as plt
tf.random.set_seed(42) 
from alive_progress import alive_bar
import time
from keras import backend as K
import pickle as pkl
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
scaler = MinMaxScaler()
categorical_cross_entropy = tf.keras.losses.CategoricalCrossentropy()

# Dataset Defender

In [ ]:
data=pd.read_csv('./dataset/Data_defense/data_defense_defender1.csv')
data_set_defense_adv=data.loc[data['label'] == 1]
data_set_defense_clean=data.loc[data['label'] == 0]

train_defender_adv, test_defender_adv = train_test_split(data_set_defense_adv,shuffle=False, test_size=0.25)
train_defender_clean, test_defender_clean= train_test_split(data_set_defense_clean, shuffle=False,test_size=0.25)
train_defender=pd.concat([train_defender_adv, train_defender_clean], ignore_index = True)
test_defender=pd.concat([test_defender_adv, test_defender_clean], ignore_index = True)

X_train_defender = train_defender.drop(columns = ["label"])
y_train_defender= train_defender.label
X_test_defender= test_defender.drop(columns = ["label"])
y_test_defender= test_defender.label


# Normalization
scaler3 = MinMaxScaler()
scaler3.fit(X_train_defender.to_numpy())
X_train_defender_scaled = scaler3.transform(X_train_defender)
X_test_defender_scaled = scaler3.transform(X_test_defender)
 
# Trained  dataset
dport_train = X_train_defender[['dportWell-known', 'dportUnspecified', 'dportRegistered', 'dportDynamic/Private']]
sport_train = X_train_defender[['sportWell-known', 'sportUnspecified',
    'sportRegistered', 'sportDynamic/Private']]
proto_train = X_train_defender[['protoudp','prototcp', 'protorarp',
    'protoigmp', 'protoicmp', 'protoarp']]
rest_train = ['dur', 'pkts', 'bytes', 'spkts',
    'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate']

data_train=[]
data_train.append(dport_train)
data_train.append(sport_train)
data_train.append(proto_train)
for i in range(0,len(rest_train)):
    d = X_train_defender[[rest_train[i]]]
    data_train.append(d)
    
# Evaluation dataset
dport_test = X_test_defender[['dportWell-known', 'dportUnspecified', 'dportRegistered', 'dportDynamic/Private']]
sport_test = X_test_defender[['sportWell-known', 'sportUnspecified',
    'sportRegistered', 'sportDynamic/Private']]
proto_test = X_test_defender[['protoudp','prototcp', 'protorarp',
    'protoigmp', 'protoicmp', 'protoarp']]
rest_test = ['dur', 'pkts', 'bytes', 'spkts',
    'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate']

data_test=[]
data_test.append(dport_test)
data_test.append(sport_test)
data_test.append(proto_test)
for i in range(0,len(rest_test)):
    d = X_test_defender[[rest_test[i]]]
    data_test.append(d)

# Training 

In [4]:
#Clusters Training 
all_cluster = []
factor_of_reliability = []
for features in range(0,len(data_test)):  

    cluster =RandomForestClassifier(n_estimators = 200, n_jobs = -1, random_state=0)
    cluster.fit(data_train[features], y_train_defender)
    # for DNN
    '''scaler.fit(data_train[features].to_numpy())
    data_train_scaled = scaler.transform(data_train[features])
    data_test_scaled = scaler.transform(data_test[features]) 
    
    y_train_defender_ohe = pd.get_dummies(y_train_defender)
    y_train_defender_ohe_tf = tf.convert_to_tensor(y_train_defender_ohe, np.float32)
    y_test_defender_ohe = pd.get_dummies(y_test_defender)
    y_test_defender_ohe_tf = tf.convert_to_tensor(y_test_defender_ohe, np.float32)

    output_number = 2
    eval_metric = 'categorical_accuracy'
    activ_out = 'softmax'
    #activ_out = 'relu'
    neurons_number = 256
    lr = 0.01
    features_number = data_train_scaled.shape[1]
    cluster = tf.keras.Sequential([tf.keras.layers.Dense(neurons_number, input_shape=(features_number,), activation="relu"), tf.keras.layers.Dense(neurons_number, activation="relu"), tf.keras.layers.Dense(output_number, activation=activ_out)])
    cluster.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss = categorical_cross_entropy, metrics=[eval_metric])
    cluster.fit(x=data_train_scaled, y=y_train_defender_ohe_tf, epochs=10, batch_size=100, verbose=1)
    cluster.evaluate(x=data_test_scaled, y=y_test_defender_ohe_tf, verbose=1)''' 
          
    pred = cluster.predict(data_test[features])
    matrix= classification_report(output_dict = True,y_true=y_test_defender, y_pred=pred)
    #matrix= classification_report(output_dict = True,y_true=y_test_defender, y_pred=y_pred_vect) #For DNN 
    factor_of_reliability.append(round(matrix['0']['recall'],2))
    all_cluster.append(cluster)

c:\Users\condor\anaconda3\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\condor\anaconda3\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\condor\anaconda3\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

## Dataset Attacker

In [5]:
# Test dataset
data_attaque=pd.read_csv('./dataset/Data_defense/data_defense_attacker.csv')
data_attaque=data_attaque.drop(columns = ['category'])
X_test_attaque= data_attaque.drop(columns = ["label"])
y_test_attaque= data_attaque.label

dport_test = X_test_defender[['dportWell-known', 'dportUnspecified', 'dportRegistered', 'dportDynamic/Private']]
sport_test = X_test_defender[['sportWell-known', 'sportUnspecified',
    'sportRegistered', 'sportDynamic/Private']]
proto_test = X_test_defender[['protoudp','prototcp', 'protorarp',
    'protoigmp', 'protoicmp', 'protoarp']]
rest_test = ['dur', 'pkts', 'bytes', 'spkts',
    'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate']

data_test=[]
data_test.append(dport_test)
data_test.append(sport_test)
data_test.append(proto_test)
for i in range(0,len(rest_test)):
    d = X_test_defender[[rest_test[i]]]
    data_test.append(d)

## Bayesian Fusion

In [ ]:
cluster_pred = []
i = 0
for cluster in all_cluster:
    # Multiplication of probs by their defensive weights
    X_defender_scaled_cluster = data_test[i]
    # For DNN
    '''scaler.fit(data_test[i].to_numpy())
    X_defender_scaled_cluster = scaler.transform(data_test[i])
    def predict_prob(number):
        return [number[0],1-number[0]]
    y_pred = np.array(list(map(predict_prob, cluster.predict(X_defender_scaled_cluster))))''' 

    y_pred = cluster.predict_proba(X_defender_scaled_cluster)
    y_pred = y_pred * factor_of_reliability[i]
    cluster_pred.append(y_pred)
    i+=1
all_fusion = []

# Bayesian_fusion, sum of clusters probabilities 
all_fusion = []
preds = sum(cluster_pred)
# Normalization, for each prediction summed before, normalization in function of the class importance
for pred in preds:
    pred_array = np.array(pred)  # convert pred to a NumPy array
    fusion = pred_array / np.sum(pred_array)  # perform normalization
    all_fusion.append(fusion.tolist())  # append normalized values to all_fusion list 

guesses = []
for i in range(len(all_fusion)):
    if all_fusion[i][0]>=all_fusion[i][1]:
        guesses.append(0) 
    else :    
        guesses.append(1) 
        
print(classification_report(y_true=y_test_defender, y_pred=guesses))   

## VoteMaj Fusion

In [ ]:
#VoteMaj    
i=0
cluster_pred = []
for cluster in all_cluster:
    X_defender_scaled_cluster = data_test[i]
    # for DNN
    '''scaler.fit(data_test[i].to_numpy())
    X_defender_scaled_cluster = scaler.transform(data_test[i])
    y_pred=cluster.predict(X_defender_scaled_cluster)
    y_pred=np.argmax(y_pred,axis=1)''' 
        
    y_pred = cluster.predict(X_defender_scaled_cluster) 
    cluster_pred.append(y_pred)
    i+=1
def MajorityVoteRule(pred_instance):
    pred_instance.count(0)
    if pred_instance.count(0)> pred_instance.count(1):
        return 0
    return 1
guesses = []
for k in range(len(data_test[0])):
    pred_instance =[]
    for j in range(len(cluster_pred)):
        pred_instance.append(cluster_pred[j][k]) 
    guesses.append(MajorityVoteRule(pred_instance))
print(classification_report(y_true=y_test_defender, y_pred=guesses))   


## DEMP_SHAFER Fusion

In [ ]:
 # DEMP_SHAFER
def DempsterRule(m1,m2):
    ## extract the frame of discernment      
    sets=set(m1.keys()).union(set(m2.keys()))
    result=dict.fromkeys(sets,0)
    ## Combination process
    for i in m1.keys():
        for j in m2.keys():
            if set(str(i)).intersection(set(str(j))) == set(str(i)):
                result[i]+=m1[i]*m2[j]
            elif set(str(i)).intersection(set(str(j))) == set(str(j)):
                result[j]+=m1[i]*m2[j]      
    ## normalize the results
    f= sum(list(result.values()))
    for i in result.keys():
        result[i] /=f
    return result
def DempsterShaferEvidenceCombinationRule (ClassProbabilities, Weights):
    MassFunctionList= []
    for i in range(len(ClassProbabilities)):
        MassFunction = {"a":Weights[i]*ClassProbabilities[i][0], "b":Weights[i]*ClassProbabilities[i][1],
                        "ab":1-Weights[i]*ClassProbabilities[i][0]-Weights[i]*ClassProbabilities[i][1]}
        MassFunctionList.append(MassFunction)
    for i,mass in enumerate(MassFunctionList):
        if i == 0:
            DSresult = mass
        else: 
            DSresult = DempsterRule(DSresult,mass)
    if DSresult['a'] >= DSresult['b']:
        return 0, DSresult['a']
    else:
        return 1, DSresult['b'] 
cluster_pred = []
i = 0
for cluster in all_cluster:
    X_defender_scaled_cluster = data_test[i]
    # For DNN
    '''scaler.fit(data_test[i].to_numpy())
    X_defender_scaled_cluster = scaler.transform(data_test[i])
    def predict_prob(number):
        return [number[0],1-number[0]]
    y_pred = np.array(list(map(predict_prob, cluster.predict(X_defender_scaled_cluster))))'''
    y_pred = cluster.predict_proba(X_defender_scaled_cluster)
    cluster_pred.append(y_pred)
    i+=1
guesses = []    
for s in range(len(data_test[0])):
    loc = []
    for n in range(len(data_test)):
        loc.append(cluster_pred[n][s]) 
    guesses.append(DempsterShaferEvidenceCombinationRule(loc , factor_of_reliability ))
guesses1 = []
for i in range(len(data_test[0])):
    guesses1.append(guesses[i][0])      
print(classification_report(y_true=y_test_defender, y_pred=guesses1))   
    